In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Preparation for reading data from csv file on Google Colab
# remove when reading the csv file locally from your computer
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/HS/WIIM_DS/04_Instacart_Data_Set/Instacart')

Mounted at /content/drive


In [3]:
op = pd.read_csv('order_products__train.csv.zip')
opp = pd.read_csv('order_products__prior.csv.zip') # If you have enough RAM
aisles = pd.read_csv('aisles.csv.zip')
departments = pd.read_csv('departments.csv.zip')
orders = pd.read_csv('orders.csv.zip')
products = pd.read_csv('products.csv.zip')

In [9]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


##Exercise

Which percentage of those orders containing ProductID 13176, which are not the last order of the corresponding user, are followed by an order containing this product, too? (“followed” referring to the next order of the same user)

In [8]:
op_all = pd.concat([opp, op]) # We will use all order positions from '_train' and '_prior'

### Solution 1


#### Step1:

Extend the orders-table by a column "contains_13176"

In [14]:
opp_13176 = op_all[op_all.product_id==13176] # filter order_positions with product 13176

In [15]:
opp_13176.head()

,order_id,product_id,add_to_cart_order,reordered
30,5,13176,1,1
241,27,13176,1,1
286,29,13176,3,1
304,32,13176,3,1
341,42,13176,3,1


In [ ]:
orders[['order_id','user_id','order_number']].merge(opp_13176[['order_id','product_id']],how='left')

In [ ]:
# Left merge orders with selected order positions
orders_ext = orders[['order_id','user_id','order_number']].merge(opp_13176[['order_id','product_id']],how='left')

In [ ]:
# Those orders with no matching order position do NOT contain product 13176
orders_ext['contains_13176'] = ~orders_ext.product_id.isna()

In [ ]:
orders_ext.head()

,order_id,user_id,order_number,product_id,contains_13176
0,2539329,1,1,NaN,False
1,2398795,1,2,13176.0,True
2,473747,1,3,NaN,False
3,2254736,1,4,NaN,False
4,431534,1,5,13176.0,True


In [ ]:
# finally remove the now unnexessary column 'product_id'
orders_ext.drop(columns=['product_id'],inplace=True)
orders_ext.head()

,order_id,user_id,order_number,contains_13176
0,2539329,1,1,False
1,2398795,1,2,True
2,473747,1,3,False
3,2254736,1,4,False
4,431534,1,5,True


In [ ]:
orders_ext.head(12)

,order_id,user_id,order_number,contains_13176
0,2539329,1,1,False
1,2398795,1,2,True
2,473747,1,3,False
3,2254736,1,4,False
4,431534,1,5,True
5,3367565,1,6,False
6,550135,1,7,False
7,3108588,1,8,False
8,2295261,1,9,False
9,2550362,1,10,False


#### Step 2:

Join each order with its successor order

In [ ]:
orders_succ = orders_ext.copy()
orders_succ.order_number = orders_succ.order_number-1
# This table states for each (user_id, order_number)-pair, which values
# the columns 'order_id' and 'contains_13176' had IN THE SUCCESSOR ORDER of the same user

In [ ]:
orders_succ.head()

,order_id,user_id,order_number,contains_13176
0,2539329,1,0,False
1,2398795,1,1,True
2,473747,1,2,False
3,2254736,1,3,False
4,431534,1,4,True


In [ ]:
orders_ext.head()

,order_id,user_id,order_number,contains_13176
0,2539329,1,1,False
1,2398795,1,2,True
2,473747,1,3,False
3,2254736,1,4,False
4,431534,1,5,True


In [ ]:
orders2 = pd.merge(orders_ext, orders_succ, on=['user_id', 'order_number'], suffixes=('','_succ'))
# Note that orders with no successor have been removed by the inner join
orders2.head(20)

,order_id,user_id,order_number,contains_13176,order_id_succ,contains_13176_succ
0,2539329,1,1,False,2398795,True
1,2398795,1,2,True,473747,False
2,473747,1,3,False,2254736,False
3,2254736,1,4,False,431534,True
4,431534,1,5,True,3367565,False
5,3367565,1,6,False,550135,False
6,550135,1,7,False,3108588,False
7,3108588,1,8,False,2295261,False
8,2295261,1,9,False,2550362,False
9,2550362,1,10,False,1187899,False


#### Step 3

Count

In [ ]:
orders2[['contains_13176','contains_13176_succ','order_id']].groupby(['contains_13176','contains_13176_succ']).count()

order_id
contains_13176 contains_13176_succ          
False          False                 2660879
               True                   174545
True           False                  178223
               True                   201227

In [ ]:
c = orders2.contains_13176.sum()
c

379450

In [ ]:
cc = (orders2.contains_13176 & orders2.contains_13176_succ).sum()
cc

201227

In [ ]:
print("The Answer to exercise 1 is ", cc/c)

The Answer to exercise 1 is  0.5303122941098959


#### Variant 2b (with shift)

In [ ]:
orders2b = orders_ext.copy()

In [ ]:
orders2b.head(12)

,order_id,user_id,order_number,contains_13176
0,2539329,1,1,False
1,2398795,1,2,True
2,473747,1,3,False
3,2254736,1,4,False
4,431534,1,5,True
5,3367565,1,6,False
6,550135,1,7,False
7,3108588,1,8,False
8,2295261,1,9,False
9,2550362,1,10,False


In [ ]:
orders2b['succ_contains_13176'] = orders2b.contains_13176.shift(-1)
orders2b.head(12)

,order_id,user_id,order_number,contains_13176,succ_contains_13176
0,2539329,1,1,False,True
1,2398795,1,2,True,False
2,473747,1,3,False,False
3,2254736,1,4,False,True
4,431534,1,5,True,False
5,3367565,1,6,False,False
6,550135,1,7,False,False
7,3108588,1,8,False,False
8,2295261,1,9,False,False
9,2550362,1,10,False,False


There is one problem: The last order of each user gets a value in "succ_contains_13176" based on the next users orders. This does not make sense:

In [ ]:
# Drop the last orders of each user
orders2b = orders2b[(orders2b.user_id == orders2b.user_id.shift(-1))]
orders2b.head(12)

,order_id,user_id,order_number,contains_13176,succ_contains_13176
0,2539329,1,1,False,True
1,2398795,1,2,True,False
2,473747,1,3,False,False
3,2254736,1,4,False,True
4,431534,1,5,True,False
5,3367565,1,6,False,False
6,550135,1,7,False,False
7,3108588,1,8,False,False
8,2295261,1,9,False,False
9,2550362,1,10,False,False


In [ ]:
orders2b[['contains_13176','succ_contains_13176','order_id']].groupby(['contains_13176','succ_contains_13176']).count()

order_id
contains_13176 succ_contains_13176          
False          False                 2660879
               True                   174545
True           False                  178223
               True                   201227

In [ ]:
c = orders2b.contains_13176.sum()
cc = (orders2b.contains_13176 & orders2b.succ_contains_13176).sum()
print("The Answer to exercise 1 is ", cc/c)

The Answer to exercise 1 is  0.5303122941098959


#### Variant 2c (shift and groupby)

In [ ]:
orders2c = orders_ext.copy()
orders2c.head(12)

,order_id,user_id,order_number,contains_13176
0,2539329,1,1,False
1,2398795,1,2,True
2,473747,1,3,False
3,2254736,1,4,False
4,431534,1,5,True
5,3367565,1,6,False
6,550135,1,7,False
7,3108588,1,8,False
8,2295261,1,9,False
9,2550362,1,10,False


In [ ]:
orders2c['succ_contains_13176'] = orders2c[['user_id','contains_13176']].groupby(['user_id']).contains_13176.shift(-1)
orders2c.head(12)

,order_id,user_id,order_number,contains_13176,succ_contains_13176
0,2539329,1,1,False,True
1,2398795,1,2,True,False
2,473747,1,3,False,False
3,2254736,1,4,False,True
4,431534,1,5,True,False
5,3367565,1,6,False,False
6,550135,1,7,False,False
7,3108588,1,8,False,False
8,2295261,1,9,False,False
9,2550362,1,10,False,False


In [ ]:
h = orders2c[['contains_13176','succ_contains_13176','order_id']].groupby(['contains_13176','succ_contains_13176']).count()
h

order_id
contains_13176 succ_contains_13176          
False          False                 2660879
               True                   174545
True           False                  178223
               True                   201227

In [ ]:
c = (orders2c.contains_13176 & ~orders2c.succ_contains_13176.isna()).sum()
cc = (orders2c.contains_13176 & orders2c.succ_contains_13176).sum()
print("The Answer to exercise 1 is ", cc/c)

The Answer to exercise 1 is  0.5303122941098959


In [ ]:
174545/(174545+2660879)

0.06155869457266356